In [1]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/Users/yaqiongzhang/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#Import Keras.
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation, BatchNormalization
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
#define our model
def getModel():
    #Building the model
    gmodel=Sequential()
    #Conv Layer 1
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    gmodel.add(BatchNormalization())
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
    gmodel.add(BatchNormalization())
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
    gmodel.add(BatchNormalization())
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
    gmodel.add(BatchNormalization())
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))
    
    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())

    #Dense Layers
    gmodel.add(Dense(512))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Dense Layer 2
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))

    mypotim=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    gmodel.compile(loss='binary_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])
    gmodel.summary()
    return gmodel


def get_callbacks(filepath, patience):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

file_path = "model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=10)

In [3]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [4]:
X_train=np.load('X_train.npy')
target_train=np.load('target_train.npy')
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train[:1604], train_size=0.9)

In [5]:
import os
gmodel=getModel()
gmodel.fit_generator(datagen.flow(X_train_cv, y_train_cv, batch_size=24),
                    steps_per_epoch=len(X_train_cv) / 24, 
                    epochs=100,
                    validation_data=(X_valid, y_valid),
                    validation_steps=len(X_valid) / 24,
                    callbacks=callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 73, 73, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 73, 73, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 71, 71, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        36928     
__________

/Users/yaqiongzhang/anaconda/lib/python3.6/site-packages/keras/preprocessing/image.py:536: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/Users/yaqiongzhang/anaconda/lib/python3.6/site-packages/keras/preprocessing/image.py:544: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


61/60 [==============================] - 121s 2s/step - loss: 3.7944 - acc: 0.5813 - val_loss: 0.6803 - val_acc: 0.6025
Epoch 2/100
61/60 [==============================] - 122s 2s/step - loss: 1.3307 - acc: 0.6195 - val_loss: 0.7671 - val_acc: 0.4969
Epoch 3/100
61/60 [==============================] - 123s 2s/step - loss: 0.8152 - acc: 0.6530 - val_loss: 0.8429 - val_acc: 0.5901
Epoch 4/100
61/60 [==============================] - 124s 2s/step - loss: 0.6818 - acc: 0.6407 - val_loss: 1.2930 - val_acc: 0.5466
Epoch 5/100
61/60 [==============================] - 124s 2s/step - loss: 0.6829 - acc: 0.6558 - val_loss: 0.8873 - val_acc: 0.5466
Epoch 6/100
61/60 [==============================] - 125s 2s/step - loss: 0.6282 - acc: 0.6728 - val_loss: 1.9262 - val_acc: 0.5466
Epoch 7/100
61/60 [==============================] - 125s 2s/step - loss: 0.5787 - acc: 0.6891 - val_loss: 1.2704 - val_acc: 0.5404
Epoch 8/100
61/60 [==============================] - 605s 10s/step - loss: 0.5441 - acc: